In [ ]:
%load_ext autoreload
%autoreload 2

import env

import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta

# --- 1. Загрузка конфигурации ---
service = env.get_gservice()

if service:
    df_sheet = env.read_df_from_spreadsheet(service, env.SHEET_ID, env.SHEET_NAME)
    print("Данные из Google Sheets загружены")
else:
    raise ConnectionError("Не удалось подключиться к Google API")

RS_TABLE_P7 = 'incent_opex_check_p7'
RS_SCHEMA_P7 = 'ma_data'
ALERT_NAME = "02-incent.p7"

try:
    config_row = df_sheet[df_sheet['name'] == ALERT_NAME].iloc[0]
except IndexError:
    raise ValueError(f"Алерт '{ALERT_NAME}' не найден в Google Sheet")

if config_row['active_flag'] != 'Enabled':
    print(f"Алерт '{ALERT_NAME}' отключен. Пропуск.")
else:
    print(f"Запуск алерта '{ALERT_NAME}'...")

# --- 2. Парсинг параметров ---
ALERT_ACTIVE_FLAG = config_row['active_flag']
THRESHOLD_WARNING = int(config_row['threshold_fixed'])        # Порог для WARNING
THRESHOLD_CRITICAL = int(config_row['threshold_fixed_crit'])  # Порог для CRITICAL
ALERT_CATEGORY = config_row['metric_crit_category']

# Хелпер для SQL списков
def to_sql_list(items):
    if not isinstance(items, list):
        items = [items] 
    if not items:
        return "()"
    
    formatted = []
    for x in items:
        if isinstance(x, str):
            formatted.append(f"'{x}'") 
        else:
            formatted.append(str(x))   
            
    return f"({', '.join(formatted)})"

try:
    # Загружаем JSON настроек
    params = json.loads(config_row['config_json'])
    
    CONFIG_PARTNERS = to_sql_list(params['partner_id'])
    
except json.JSONDecodeError as e:
    raise ValueError(f"Ошибка JSON в ячейке config_json: {e}")
except KeyError as e:
    raise ValueError(f"В JSON отсутствует обязательный ключ: {e}")

print(f"Настройки: THRESHOLD_WARNING={THRESHOLD_WARNING}, THRESHOLD_CRITICAL={THRESHOLD_CRITICAL}")
print(f"Partners: {CONFIG_PARTNERS}")


# --- 3. Функция расчёта периода дат ---

def get_check_period():
    """
    Возвращает (start_date, end_date) - период Пт-Чт (7 дней)
    с учётом 7-дневного лага для созревания payers_7
    
    Логика:
    - Находим последний четверг, для которого прошло СТРОГО БОЛЬШЕ 7 дней
      (т.е. >= 8 дней между четвергом и сегодня)
    - Это гарантирует, что payers_7 полностью "созрели"
    - Период: пятница (start) - четверг (end)
    """
    today = datetime.now().date()
    
    # Находим последний четверг
    # weekday: Mon=0, Tue=1, Wed=2, Thu=3, Fri=4, Sat=5, Sun=6
    days_since_thursday = (today.weekday() - 3) % 7
    if days_since_thursday == 0:
        days_since_thursday = 7  # Если сегодня четверг, берём предыдущий
    last_thursday = today - timedelta(days=days_since_thursday)
    
    # Требуем СТРОГО БОЛЬШЕ 7 дней (>7, т.е. >= 8)
    # Это значит: today - period_end >= 8
    days_passed = (today - last_thursday).days
    if days_passed <= 8:
        last_thursday -= timedelta(days=7)
    
    # Период: пятница (start) - четверг (end)
    period_end = last_thursday
    period_start = period_end - timedelta(days=6)
    
    return period_start, period_end


# --- 4. Функция определения типа алерта ---

def get_alert_type(payers_count):
    """
    Определяет тип алерта на основе количества плательщиков.
    CRITICAL < WARNING (более строгий порог)
    
    Returns: 'CRITICAL', 'WARNING', or None
    """
    if payers_count < THRESHOLD_CRITICAL:
        return 'CRITICAL'
    elif payers_count < THRESHOLD_WARNING:
        return 'WARNING'
    else:
        return None


# --- 5. Основная функция проверки ---

def run_payers_check():
    
    period_start, period_end = get_check_period()
    
    print(f"\n--- Checking Payers7 ---")
    print(f"Период: {period_start} - {period_end}")
    
    # SQL Запрос
    sql_query = f"""
    SELECT
        app_short,
        operation_segment_nm,
        partner_id,
        COUNT(DISTINCT CASE WHEN payers_7_cnt = 1 THEN plr_id END) as payers_count,
        SUM(installs_cnt) as total_installs
    FROM core.base_metrics
    WHERE
        install_dt BETWEEN '{period_start}' AND '{period_end}'
        AND partner_id IN {CONFIG_PARTNERS}
    GROUP BY app_short, operation_segment_nm, partner_id
    """
    
    df = env.execute_sql(sql_query)
    
    if df.empty:
        print("  >> Нет данных за указанный период.")
        return df
    
    print(f"  >> Получено строк: {len(df)}")
    
    # Приводим типы
    df['payers_count'] = pd.to_numeric(df['payers_count'], errors='coerce').fillna(0).astype(int)
    df['total_installs'] = pd.to_numeric(df['total_installs'], errors='coerce').fillna(0).astype(int)
    
    # Добавляем метаданные
    df['period_start'] = period_start
    df['period_end'] = period_end
    df['threshold_warning'] = THRESHOLD_WARNING
    df['threshold_critical'] = THRESHOLD_CRITICAL
    
    # Определяем тип алерта для каждой строки
    df['alert_type'] = df['payers_count'].apply(get_alert_type)
    
    # Флаг наличия алерта (любого типа)
    df['is_alert'] = df['alert_type'].notna()
    
    return df


# --- 6. Запуск проверки ---

df_result = run_payers_check()

if not df_result.empty:
    alerts_df = df_result[df_result['is_alert'] == True].copy()
    
    if not alerts_df.empty:
        alerts_df['check_name'] = ALERT_NAME
        alerts_df['metric_crit_category'] = ALERT_CATEGORY
        alerts_df['date'] = datetime.now()
        
        # Подсчёт по типам
        critical_count = (alerts_df['alert_type'] == 'CRITICAL').sum()
        warning_count = (alerts_df['alert_type'] == 'WARNING').sum()
        
        print(f"\n[{ALERT_CATEGORY.upper()}] Алерты найдены: {len(alerts_df)} (CRITICAL: {critical_count}, WARNING: {warning_count})")
        
        # --- 7. Запись в Redshift ---
        db_cols = [
            'date', 'check_name', 'metric_crit_category',
            'app_short', 'operation_segment_nm', 'partner_id',
            'payers_count', 'total_installs', 
            'threshold_warning', 'threshold_critical',
            'alert_type', 'is_alert',
            'period_start', 'period_end'
        ]
        
        df_to_write = alerts_df[db_cols].copy()
        
        if not df_to_write.empty:
            print(f"Запись {len(df_to_write)} строк в Redshift...")
            env.insert_table_into_rs(df_to_write, RS_TABLE_P7, RS_SCHEMA_P7, 10000)
            print("Успешно записано.")
        
        # --- 8. Slack нотификации ---
        if ALERT_ACTIVE_FLAG != 'Enabled':
            print(f"Нотификация '{ALERT_NAME}' отключена.")
        else:
            # Индикаторы для разных типов алертов
            ICON_CRITICAL = ":red_circle:"           # Красный для CRITICAL
            ICON_WARNING = ":large_yellow_circle:"   # Жёлтый для WARNING

            # Группируем по app + partner для формирования сообщений
            unique_targets = alerts_df[['app_short', 'partner_id']].drop_duplicates()

            for _, target_row in unique_targets.iterrows():
                t_app = target_row['app_short']
                t_partner = target_row['partner_id']

                subset = alerts_df[
                    (alerts_df['app_short'] == t_app) &
                    (alerts_df['partner_id'] == t_partner)
                ]

                # Определяем общий индикатор: CRITICAL если есть хотя бы один CRITICAL
                has_critical = (subset['alert_type'] == 'CRITICAL').any()
                header_icon = ICON_CRITICAL if has_critical else ICON_WARNING

                # Основное сообщение (в канал)
                main_message = f"{header_icon} INCENT.OpEx - {ALERT_NAME} ({ALERT_CATEGORY}): *{t_app.upper()} (partner: {t_partner})*"

                # Детали по сегментам (в тред)
                thread_lines = []
                subset_sorted = subset.sort_values(by='alert_type', ascending=True)

                for _, row in subset_sorted.iterrows():
                    segment = row['operation_segment_nm']
                    payers = row['payers_count']
                    alert_type = row['alert_type']

                    if alert_type == 'CRITICAL':
                        icon = ICON_CRITICAL
                        threshold = THRESHOLD_CRITICAL
                    else:
                        icon = ICON_WARNING
                        threshold = THRESHOLD_WARNING

                    line = f"{icon} [{alert_type}] {segment}: {payers} payers (threshold: {threshold})"
                    thread_lines.append(line)

                try:
                    slack = env.SlackNotifier("incent_notifications")

                    # 1. Отправляем основное сообщение (в канал)
                    thread_ts = slack.send_message(main_message)

                    # 2. Отправляем детали по сегментам (в тред)
                    if thread_lines:
                        thread_message = "\n".join(thread_lines)
                        slack.send_message(thread_message, thread_ts=thread_ts)

                except Exception as e:
                    print(f"Error sending to Slack: {e}")
        
        # --- 9. Вывод отчёта в Jupyter ---
        display_cols = [
            'date', 'check_name', 'metric_crit_category',
            'app_short', 'operation_segment_nm', 'partner_id',
            'payers_count', 'total_installs', 
            'threshold_warning', 'threshold_critical',
            'alert_type',
            'period_start', 'period_end'
        ]
        
        # Сортировка: CRITICAL сверху
        alerts_display = alerts_df[display_cols].sort_values(by='alert_type', ascending=True)
        styled_df = alerts_display.style.hide(axis='index')
        display(styled_df)
        
    else:
        print(f"Алертов не найдено. Все сегменты выше порогов (WARNING: {THRESHOLD_WARNING}, CRITICAL: {THRESHOLD_CRITICAL}).")
else:
    print("Нет данных для анализа.")